In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
f = open(r'images/templeSparseRing/templeSR_par.txt','r')
assert f is not None
n = int(f.readline())

#First Image
l = f.readline().split()
im1_fn = l[0]
k1 = np.array([float(i) for i in l[1:10]]).reshape((3,3))
r1 = np.array([float(i) for i in l[10:19]]).reshape((3,3))
t1 = np.array([float(i) for i in l[19:22]]).reshape((3,1))

#Second Image
l = f.readline().split()
im2_fn = l[0]
k2 = np.array([float(i) for i in l[1:10]]).reshape((3,3))
r2 = np.array([float(i) for i in l[10:19]]).reshape((3,3))
t2 = np.array([float(i) for i in l[19:22]]).reshape((3,1))

In [3]:
im1 = cv.imread(r'images/templeSparseRing/'+im1_fn,cv.IMREAD_COLOR)
im2 = cv.imread(r'images/templeSparseRing/'+im2_fn,cv.IMREAD_COLOR)
assert im1 is not None
assert im2 is not None

Q1

In [13]:
sift = cv.xfeatures2d.SIFT_create()
kp1,desc1 = sift.detectAndCompute(im1,None)
kp2,desc2 = sift.detectAndCompute(im2,None)

FLANN_INDEX_KDTREE = 1

indx_para = dict(algorithm = FLANN_INDEX_KDTREE,trees = 5)
search_para = dict(checks = 100)
flann = cv.FlannBasedMatcher(indx_para,search_para)
matches = flann.knnMatch(desc1,desc2, k= 2)

good = []
pts1,pts2 = [],[]

for i,(m,n) in enumerate(matches):
    if m.distance < 0.7 *n.distance:
        good.append(m)
        pts1.append(kp1[m.queryIdx].pt)
        pts2.append(kp2[m.trainIdx].pt)

pts1 = np.array(pts1)
pts2 = np.array(pts2)


Q2

In [20]:
F,mask = cv.findFundamentalMat(pts1,pts2,cv.FM_RANSAC)
E = k2.T @ F@ k1

print("Fundamental Matrix : ",F)
print("Essential Matrix : ", E)

Fundamental Matrix :  [[ 1.19353197e-06  1.48128487e-05 -2.65668422e-02]
 [-8.37167541e-06  6.34793204e-07  2.04080864e-03]
 [ 2.41439516e-02 -5.73622910e-03  1.00000000e+00]]
Essential Matrix :  [[ 2.75898779e+00  3.43654884e+01 -3.42837514e+01]
 [-1.94221058e+01  1.47803397e+00 -5.08742503e-01]
 [ 3.41148335e+01 -1.68046954e+00 -1.62748485e-02]]


Q3

In [18]:
retval, R,t ,mask = cv.recoverPose(E,pts1,pts2,k1)

Q4

In [21]:
R_t_1 = np.concatenate((r1,t1),axis=1)

R2 = r1 @ R
t2 = r1 @ t

R_t_2 = np.concatenate((R2,t2),axis=1)
P2_ = k2 @ R_t_2

print("Camera Matrix : ",P2_)

Camera Matrix :  [[ 1.56140182e+02  1.53317827e+03 -1.67326558e+02 -9.66850517e+02]
 [ 1.53102041e+03 -1.25962559e+02 -1.71538765e+02  1.56694615e+02]
 [ 5.65837070e-02  8.28361136e-02 -9.94955508e-01  6.45008519e-01]]
